# Checkpoint Three: Cleaning Data

Now you are ready to clean your data. Before starting coding, provide the link to your dataset below.

My dataset:

Import the necessary libraries and create your dataframe(s).
My dataset is https://www.kaggle.com/datasets/raghadalharbi/breast-cancer-gene-expression-profiles-metabric 

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Creating a dataframe
br_df = pd.read_csv("METABRIC_RNA_Mutation.csv",low_memory = False)

## Missing Data

Test your dataset for missing data and handle it as needed. Make notes in the form of code comments as to your thought process.


In [2]:
# Checking rows and columns of the dataframe
br_df.shape

(1904, 693)

In [3]:
# Checking for null values and percentage of nulls in descending order

counts_null = br_df.isna().sum().sort_values(ascending = False)
percentage_null = round(br_df.isna().mean() * 100, 1).sort_values(ascending=False)
null_values = pd.concat([counts_null, percentage_null], axis=1, keys=["count_null", "percent_null"])
null_values.head(15)

,count_null,percent_null
tumor_stage,501,26.3
3-gene_classifier_subtype,204,10.7
primary_tumor_laterality,106,5.6
neoplasm_histologic_grade,72,3.8
cellularity,54,2.8
mutation_count,45,2.4
er_status_measured_by_ihc,30,1.6
type_of_breast_surgery,22,1.2
tumor_size,20,1.1
cancer_type_detailed,15,0.8


In [4]:

# will remove the columns with top 2 null counts tumor stage' and '3-gene_classifier_subtype'
# Tumor stage has correlation with Tumor size, so keeping one will be enough for now.
#Also the 3-gene-classifier has variabales from other columns(her and er status)

br_df = br_df.drop(['tumor_stage', '3-gene_classifier_subtype', 'primary_tumor_laterality', 
                  'er_status_measured_by_ihc'], axis=1)

# dropped 2 other columns not needed which have null values

In [5]:
# will drop the rows with null values as accuracy is valued in healthcare data and sample size is large enough

cl_br_df = br_df.dropna(subset = ['cellularity', 'mutation_count', 'type_of_breast_surgery',
                                  'neoplasm_histologic_grade','tumor_size', 'cancer_type_detailed',
                                  'tumor_other_histologic_subtype',
                                  'oncotree_code', 'death_from_cancer' ], axis = 0, how = 'any' )

In [6]:
# Checking to see if null values have been removed
cl_br_df.isna().sum().sum()

0

## Irregular Data

Detect outliers in your dataset and handle them as needed. Use code comments to make notes about your thought process.

In [7]:
# In my eda check point 2, I have detected outliers in some columns, including number of positive lymph nodes
#and tumor diameter. I will keep these as it is is clinical data.

## Unnecessary Data

Look for the different types of unnecessary data in your dataset and address it as needed. Make sure to use code comments to illustrate your thought process.

In [8]:
# As I am not dealing with the gene data for my analyis, will drop them.
#  will also drop 'patient id' and 'her2_status_measured_by_snp6' as they are not needed for my analysis 

In [9]:
features_to_drop = cl_br_df.columns[27:] # dropping non clinical attributes
cl_br_df = cl_br_df.drop(features_to_drop, axis=1)
cl_br_df.drop(['patient_id', 'her2_status_measured_by_snp6'],axis=1,inplace=True)

In [10]:
# Info of cleaned data frame so far
cl_br_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1697 entries, 1 to 1903
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   age_at_diagnosis                1697 non-null   float64
 1   type_of_breast_surgery          1697 non-null   object 
 2   cancer_type                     1697 non-null   object 
 3   cancer_type_detailed            1697 non-null   object 
 4   cellularity                     1697 non-null   object 
 5   chemotherapy                    1697 non-null   int64  
 6   pam50_+_claudin-low_subtype     1697 non-null   object 
 7   cohort                          1697 non-null   float64
 8   er_status                       1697 non-null   object 
 9   neoplasm_histologic_grade       1697 non-null   float64
 10  her2_status                     1697 non-null   object 
 11  tumor_other_histologic_subtype  1697 non-null   object 
 12  hormone_therapy                 16

In [11]:
# Checking for any duplicate rows
cl_br_df.duplicated().sum()


0

## Inconsistent Data

Check for inconsistent data and address any that arises. As always, use code comments to illustrate your thought process.

In [12]:
# Checking for incosistent data
cl_br_df.head(5)

,age_at_diagnosis,type_of_breast_surgery,cancer_type,cancer_type_detailed,cellularity,chemotherapy,pam50_+_claudin-low_subtype,cohort,er_status,neoplasm_histologic_grade,...,lymph_nodes_examined_positive,mutation_count,nottingham_prognostic_index,oncotree_code,overall_survival_months,overall_survival,pr_status,radio_therapy,tumor_size,death_from_cancer
1,43.19,BREAST CONSERVING,Breast Cancer,Breast Invasive Ductal Carcinoma,High,0,LumA,1.0,Positive,3.0,...,0.0,2.0,4.020,IDC,84.633333,1,Positive,1,10.0,Living
2,48.87,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,High,1,LumB,1.0,Positive,2.0,...,1.0,2.0,4.030,IDC,163.700000,0,Positive,0,15.0,Died of Disease
3,47.68,MASTECTOMY,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,Moderate,1,LumB,1.0,Positive,2.0,...,3.0,1.0,4.050,MDLC,164.933333,1,Positive,1,25.0,Living
4,76.97,MASTECTOMY,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,High,1,LumB,1.0,Positive,3.0,...,8.0,2.0,6.080,MDLC,41.366667,0,Positive,1,40.0,Died of Disease
5,78.77,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,Moderate,0,LumB,1.0,Positive,3.0,...,0.0,4.0,4.062,IDC,7.800000,0,Positive,1,31.0,Died of Disease


In [13]:
# Will round the decimal columns in 'Overall Survival months' and 'nottingham prognostic index'
cl_br_df['overall_survival_months'] = cl_br_df['overall_survival_months'].apply(lambda x: round(x, 2))
cl_br_df['nottingham_prognostic_index'] = cl_br_df['nottingham_prognostic_index'].apply(lambda x: round(x, 2))


In [14]:
# Renaming columns for easier readability
cl_br_df.rename(columns={'pam50_+_claudin-low_subtype': "pam50_claudin_low_subtype", 'er_status': 'estrogen_status',
                                    'pr_status':'progesterone_status'},inplace=True
                          )


In [15]:
# displaying max columns to check again
pd.set_option('display.max_columns', None)
cl_br_df.head(50)

,age_at_diagnosis,type_of_breast_surgery,cancer_type,cancer_type_detailed,cellularity,chemotherapy,pam50_claudin_low_subtype,cohort,estrogen_status,neoplasm_histologic_grade,her2_status,tumor_other_histologic_subtype,hormone_therapy,inferred_menopausal_state,integrative_cluster,lymph_nodes_examined_positive,mutation_count,nottingham_prognostic_index,oncotree_code,overall_survival_months,overall_survival,progesterone_status,radio_therapy,tumor_size,death_from_cancer
1,43.19,BREAST CONSERVING,Breast Cancer,Breast Invasive Ductal Carcinoma,High,0,LumA,1.0,Positive,3.0,Negative,Ductal/NST,1,Pre,4ER+,0.0,2.0,4.02,IDC,84.63,1,Positive,1,10.0,Living
2,48.87,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,High,1,LumB,1.0,Positive,2.0,Negative,Ductal/NST,1,Pre,3,1.0,2.0,4.03,IDC,163.70,0,Positive,0,15.0,Died of Disease
3,47.68,MASTECTOMY,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,Moderate,1,LumB,1.0,Positive,2.0,Negative,Mixed,1,Pre,9,3.0,1.0,4.05,MDLC,164.93,1,Positive,1,25.0,Living
4,76.97,MASTECTOMY,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,High,1,LumB,1.0,Positive,3.0,Negative,Mixed,1,Post,9,8.0,2.0,6.08,MDLC,41.37,0,Positive,1,40.0,Died of Disease
5,78.77,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,Moderate,0,LumB,1.0,Positive,3.0,Negative,Ductal/NST,1,Post,7,0.0,4.0,4.06,IDC,7.80,0,Positive,1,31.0,Died of Disease
6,56.45,BREAST CONSERVING,Breast Cancer,Breast Invasive Ductal Carcinoma,Moderate,1,LumB,1.0,Positive,2.0,Negative,Ductal/NST,1,Post,3,1.0,4.0,4.02,IDC,164.33,1,Positive,1,10.0,Living
7,89.08,BREAST CONSERVING,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,Moderate,0,claudin-low,1.0,Positive,2.0,Negative,Mixed,1,Post,3,1.0,1.0,4.06,MDLC,99.53,0,Negative,1,29.0,Died of Other Causes
8,86.41,BREAST CONSERVING,Breast Cancer,Breast Invasive Ductal Carcinoma,Moderate,0,LumB,1.0,Positive,3.0,Negative,Ductal/NST,1,Post,9,1.0,4.0,5.03,IDC,36.57,0,Negative,1,16.0,Died of Other Causes
9,84.22,MASTECTOMY,Breast Cancer,Breast Invasive Lobular Carcinoma,High,0,Her2,1.0,Positive,2.0,Negative,Lobular,0,Post,3,0.0,5.0,3.06,ILC,36.27,0,Negative,0,28.0,Died of Disease
10,85.49,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,Moderate,0,LumA,1.0,Positive,2.0,Negative,Ductal/NST,1,Post,3,0.0,1.0,3.04,IDC,132.03,0,Positive,1,22.0,Died of Disease


In [16]:
# Exporting the cleaned data set
cl_br_df.to_csv("clean_metabric.csv")

## Summarize Your Results

Make note of your answers to the following questions.

1. Did you find all four types of dirty data in your dataset?
Yes, I did find all the four types.

2. Did the process of cleaning your data give you new insights into your dataset?
Yes, I was able to see the details of classification subtypes and noticed that some columns had mixed data types.

3. Is there anything you would like to make note of when it comes to manipulating the data and making visualizations?
I will have to create a calculated field for my combination of treatments for further manipulation and also explore options for the same in Tableau.



